In [ ]:
# First import the library
import pyrealsense2 as rs
# Import Numpy for easy array manipulation
import numpy as np
# Import OpenCV for easy image rendering
import cv2



In [ ]:
# Create a pipeline
pipeline = rs.pipeline()

#Create a config and configure the pipeline to stream
#  different resolutions of color and depth streams
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
profile = pipeline.start(config)


curr_frame = 0

In [ ]:
        # Getting the depth sensor's depth scale (see rs-align example for explanation)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)

# We will be removing the background of objects more than
#  clipping_distance_in_meters meters away
clipping_distance_in_meters = 1 #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale

In [ ]:
try:
    while True:
        # Get frameset of color and depth
        frames = pipeline.wait_for_frames()
        # frames.get_depth_frame() is a 640x360 depth image

        # Align the depth frame to color frame
        aligned_frames = align.process(frames)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame = aligned_frames.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame or not color_frame:
            continue

        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Remove background - Set pixels further than clipping_distance to grey
        grey_color = 153
        depth_image_3d = np.dstack((depth_image,depth_image,depth_image)) #depth image is 1 channel, color is 3 channels
        bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), grey_color, color_image)
        
        # detect objects using bounding rectangle:===
        # function to detect objects,..................
---------------------------------------------------------------------------------------------------------
        #converting the image into grayscale as  
        gray = cv2.cvtcolor(bg_removed,cvt_bgr2gray)
        cv2.imshow('grey image',grey)
        # using blurr
        im_blur = cv2.GaussianBlur(gray,(5,5),0)
        # finding canny edges
        threshold = 10                                 ## try providing val as lower value such as 10
        canny_output = cv2.Canny(im_blur, threshold, threshold * 3+30)
        contours, hierarchy = cv2.findContours(canny_output, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)

        # finding the largest contour according to size outpu of canny edges is a black backgroud
        sorted_contours = sorted(contours, key = cv2.contourArea, reverse = True)

        #bounding rectangle for the largest contours (only for testing later will be commmented)====
        x,y,w,h = cv2.boundingRect(sorted_contours[0])
        cv2.rectangle(color_im,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.imshow('bounding rectangle', color_im)

        # now we have drawn the bounding rectangle let us use depth image 
        cropped = color_im[x:x+w,y:y+h]
        #let us try to design an algo to detect wheather a cylindrical or prism
        #intitially to detect either a cylinder or a prism on
      #----------------------------------------------------------------------------------------------------------------  
                                                        #ALGO 1
            # find the center point 
            # convert image to grayscale image
        gray_image = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)

        # convert the grayscale image to binary image
        ret,gray_image = cv2.threshold(gray_image,127,255,0)

        # calculate moments of binary image
        M = cv2.moments(gray_image)

        # calculate x,y coordinate of center
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])

        # take three points between the center and the edge
        arr = []
        for i in range(5):
            a = cX- i*w/2*4
            #compute the depth parameter
            dist = depth_image.get_distance(a,cY)
            arr.append(dist) 

        # analyzing arr
        dpt= arr[4]-arr[0]
        ar2= []
        for i in range(cX,x,-1):
            dist = depth_image.get_distance(i,cY)
            ar2.append(dist) 

            
        import collections 
        import numpy as np
        from scipy.signal import argrelextrema
        arr_sor = sorted(arr, reverse = false)
        if collections.Counter(arr_sor) == collections.Counter(arr): 
            print ("The object is not a cylinder") 
            if ( dpt<1.1 * w/2 )and( dpt > 0.9 * w/2 ):
                print ("The object is  a cylinder") 
        else : 
                print ("The object is not a cylinder") 
                if (dpt < 0.01* w/2) and (dpt > -0.01 * w/2):  # the given probablity value can be adjusted for better estimates
                    print ( "the object can be used as an rectagular prism with width = "+ string(w))
                    
                if argrelextrema(ar2, np.greater).shape[0] > = 1:
                    print(" use a bounding cuboidal frame")
                    
                
                
                
                
                
                
                
  # Render images
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        images = np.hstack((color_im, depth_colormap))
        cv2.namedWindow('Align Example', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('Align Example', images)
        key = cv2.waitKey(1)
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
finally:
    pipeline.stop()
    
    
    
    
    
    
    